In [1]:
import shapely.geometry
import pyproj
import json

from grid import Grid
from cell import Cell
from entrie import Entrie, EntrieType 
from utils import *

In [41]:
spb_grid = Grid.make_grid_in_degrees((60.11862, 29.932493), (60.100862, 29.98))
smr_grid = Grid.make_grid_in_degrees((53.277885, 50.056253), (53.174675, 50.319944))

In [42]:
with open('./data/entries/parsed/smr.json') as file:
    etrie_data = json.load(file)

In [43]:
entries = Cell([0, 0]).entries

In [ ]:
entries[EntrieType(list(etrie_data.keys())[0])]
i = 0
for entrie_type_value, raw_entries in zip(etrie_data.keys(), etrie_data.values()):
    entrie_type = EntrieType(entrie_type_value)
    for raw_entrie in raw_entries:
        processed_entrie = Entrie(*raw_entrie)
        entries[entrie_type].append(processed_entrie)
        smr_grid.add_entrie(processed_entrie, entrie_type)
    i += 1
    print(f'{i}/{len(entries)})

In [25]:
smr_grid.find_suitable_position(50.1995781, 53.2009557)

In [47]:
r = smr_grid.get_all_entries_in_radius((53.277885, 50.056253), 10000)

In [32]:
c1 = grid.cells[0][0] + grid.cells[0][1]
c2 = grid.cells[0][2] + grid.cells[0][3]

In [40]:
distance(grid.cells[0][19].center, grid.cells[0][20].center) * 2.325

100.2975021357343

In [89]:
29.9328822447388 - 29.93366073117118

-0.000778486432380987

In [88]:
grid.cells[0][0].center, grid.cells[0][1].center

((60.14131115764205, 29.9328822447388), (60.14131115764205, 29.93366073117118))

In [26]:
midpoint(grid.cells[0][2].center, grid.cells[0][3].center)

(60.16331988210298, 30.04919449246976)

In [83]:
# Set up projections
p_ll = pyproj.Proj(init='epsg:4326')
p_mt = pyproj.Proj(init='epsg:3857') # metric; same as EPSG:900913

# Create corners of rectangle to be transformed to a grid
nw = shapely.geometry.Point((-5.0, 40.0))
se = shapely.geometry.Point((-4.999, 40.03439880201911))

stepsize = 50

# Project corners to target projection
s = pyproj.transform(p_ll, p_mt, nw.x, nw.y) # Transform NW point to 3857
e = pyproj.transform(p_ll, p_mt, se.x, se.y) # .. same for SE

# Iterate over 2D area
gridpoints = []
x = s[0]
while x < e[0]:
    y = s[1]
    row = []
    while y < e[1]:
        p = shapely.geometry.Point(pyproj.transform(p_mt, p_ll, x, y))
        row.append([p.x, p.y])
        y += stepsize * 1.314
    gridpoints.append(row)
    x += stepsize

In [85]:
distance([-5.0, 40.00045211267976], [-5.0, 40.0])

50.137436430313635

In [ ]:
gridpoints

In [63]:
distance((-5.0, 40.03439880201911), [-5.0, 40.0])

3814.68564478778